## Get log for K-means clustering

In [14]:
import requests
import csv

MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = 'bc40c9880bc730d7ee13f40323a45b23'
USER_ID = 4
COURSE_ID = 5
FORMAT = 'json'

def call_api(function, extra_params):
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': FORMAT,
        'wsfunction': function
    }
    params.update(extra_params)
    
    response = requests.post(MOODLE_URL, data=params)
    return response.json()

def main():
    # Gọi API để lấy dữ liệu gốc
    quiz_attempts = call_api('local_userlog_get_quiz_attempts', {'userid': USER_ID, 'courseid': COURSE_ID}).get('quiz_attempts', 0)
    total_time = call_api('local_userlog_get_total_quiz_time', {'userid': USER_ID, 'courseid': COURSE_ID}).get('total_quiz_time', 0)
    resource_views = call_api('local_userlog_get_resource_views', {
        'userid': USER_ID,
        'courseid': COURSE_ID,
        'objecttypes[0]': 'resource',
        'objecttypes[1]': 'hvp',
        'objecttypes[2]': 'quiz'
    }).get('resource_views', 0)
    learning_days = call_api('local_userlog_get_learning_days', {
        'userid': USER_ID,
        'courseid': COURSE_ID,
        'objecttypes[0]': 'resource',
        'objecttypes[1]': 'hvp',
        'objecttypes[2]': 'quiz'
    }).get('num_learning_days', 1)
    pass_quiz_count = call_api('local_userlog_get_pass_quiz_count_attempt', {'userid': USER_ID, 'courseid': COURSE_ID}).get('pass_quiz_count', 0)
    avg_quiz_score = call_api('local_userlog_get_avg_quiz_score', {'userid': USER_ID, 'courseid': COURSE_ID}).get('avg_quiz_score', 0.0)

    # Tính các đặc trưng
    avg_time_per_quiz = round(total_time / quiz_attempts, 2) if quiz_attempts else 0
    avg_resource_views_per_day = round(resource_views / learning_days, 2)
    quiz_success_rate = round(pass_quiz_count / quiz_attempts, 2) if quiz_attempts else 0
    print(f"[DEBUG] quiz_success_rate = {quiz_success_rate}")
    print(f"[DEBUG] quiz_attempts = {quiz_attempts}")
    print(f"[DEBUG] pass_quiz_count = {pass_quiz_count}")

    resource_vs_quiz_ratio = round(resource_views / quiz_attempts, 2) if quiz_attempts else 0

    # Dữ liệu đầu ra
    final_features = {
        'userid': USER_ID,
        'courseid': COURSE_ID,
        'avg_time_per_quiz': avg_time_per_quiz,
        'avg_resource_views_per_day': avg_resource_views_per_day,
        'quiz_success_rate': quiz_success_rate,
        'avg_quiz_score': avg_quiz_score,
        'resource_vs_quiz_ratio': resource_vs_quiz_ratio
    }

    # Ghi ra file CSV
    with open('user_features.csv', 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=final_features.keys())
        writer.writeheader()
        writer.writerow(final_features)

    print("✅ Đã ghi user_features.csv thành công.")

if __name__ == '__main__':
    main()

[DEBUG] quiz_success_rate = 0.83
[DEBUG] quiz_attempts = 30
[DEBUG] pass_quiz_count = 25
✅ Đã ghi user_features.csv thành công.


In [4]:
import csv
import random

def read_base_from_csv(filename='user_features.csv'):
    with open(filename, 'r', newline='') as f:
        reader = csv.DictReader(f)
        first_row = next(reader)
        base = {
            'avg_time_per_quiz': float(first_row['avg_time_per_quiz']),
            'avg_resource_views_per_day': float(first_row['avg_resource_views_per_day']),
            'quiz_success_rate': float(first_row['quiz_success_rate']),
            'avg_quiz_score': float(first_row['avg_quiz_score']),
            'resource_vs_quiz_ratio': float(first_row['resource_vs_quiz_ratio']),
        }
        return base

def generate_data(base, userid_start=5, count=100, courseid=5):
    data = []
    for i in range(count):
        userid = userid_start + i
        row = {
            'userid': userid,
            'courseid': courseid,
            'avg_time_per_quiz': round(base['avg_time_per_quiz'] * random.uniform(0.8, 1.2), 2),
            'avg_resource_views_per_day': round(base['avg_resource_views_per_day'] * random.uniform(0.8, 1.2), 2),
            'quiz_success_rate': round(min(1.0, base['quiz_success_rate'] * random.uniform(0.8, 1.2)), 2),
            'avg_quiz_score': round(base['avg_quiz_score'] * random.uniform(0.8, 1.2), 2),
            'resource_vs_quiz_ratio': round(base['resource_vs_quiz_ratio'] * random.uniform(0.8, 1.2), 2),
        }
        data.append(row)
    return data

def save_csv(data, filename='synthetic_user_features.csv'):
    if not data:
        return
    with open(filename, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)
    print(f"✅ Saved {len(data)} rows to {filename}")

if __name__ == '__main__':
    base = read_base_from_csv('user_features.csv')
    synthetic_data = generate_data(base, userid_start=5, count=1000)
    save_csv(synthetic_data)

✅ Saved 1000 rows to synthetic_user_features.csv


In [ ]:
import json
import requests
import csv

MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = '5243b89497861d41b63664b739e47760'
USER_ID = 4
COURSE_ID = 5
FORMAT = 'json'

def call_api(function, extra_params):
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': FORMAT,
        'wsfunction': function
    }
    params.update(extra_params)
    
    response = requests.post(MOODLE_URL, data=params)
    return response.json()

    # Gọi API để lấy dữ liệu gốc
grade_data = call_api('gradereport_user_get_grade_items', {'userid': USER_ID, 'courseid': COURSE_ID})

    # In dữ liệu dạng JSON đẹp
print(json.dumps(grade_data, indent=4, ensure_ascii=False))

{
    "exception": "webservice_access_exception",
    "errorcode": "accessexception",
    "message": "Access control exception (Access to the function gradereport_user_get_grade_items() is not allowed.\n                     There could be multiple reasons for this:\n                     1. The service linked to the user token does not contain the function.\n                     2. The service is user-restricted and the user is not listed.\n                     3. The service is IP-restricted and the user IP is not listed.\n                     4. The service is time-restricted and the time has expired.\n                     5. The token is time-restricted and the time has expired.\n                     6. The service requires a specific capability which the user does not have.\n                     7. The function is called with username/password (no user token is sent)\n                     and none of the services has the function to allow the user.\n                     These settin

In [ ]:
import json
import requests
import csv

MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = '84cffdbb9ead18d97ccc45f9889bc926'
USER_ID = 4
SECTION_ID = 42
COURSE_ID = 5
FORMAT = 'json'

def call_api(function, extra_params):
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': FORMAT,
        'wsfunction': function
    }
    params.update(extra_params)
    
    response = requests.post(MOODLE_URL, data=params)
    return response.json()

# Dữ liệu JSON để tạo quiz
quiz_data = {
    'courseid': 5,
    'quizname': 'Bài kiểm tra cuối kỳ môn Tin học - 120825 - 200',
    'userid': USER_ID,  # Thêm user ID để chỉ hiển thị quiz cho user cụ thể
    'questions[0][categoryid]': 15,
    'questions[0][easy]': 2,
    'questions[0][medium]': 1,
    'questions[0][hard]': 1
}

# Gọi API để tạo quiz
create_quiz_response = call_api('local_userlog_create_quiz_from_categories', quiz_data)
print(json.dumps(create_quiz_response, indent=4, ensure_ascii=False))

# API lấy thông tin trạng thái làm bài kiểm tra cuối cùng của 1 section nào đó
# grade_data = call_api('local_userlog_get_latest_quiz_pass_status_by_section', {'sectionid': SECTION_ID, 'userid': USER_ID})
# print(json.dumps(grade_data, indent=4, ensure_ascii=False))

# API lấy thông tin category của question bank trong 1 khoá học
# category_questionbank_data = call_api('local_userlog_get_category_questionbank', {'courseid': COURSE_ID})
# print(json.dumps(category_questionbank_data, indent=4, ensure_ascii=False))

{
    "quizid": 34,
    "cmid": 105
}
